<a href="https://colab.research.google.com/github/Brekalo/Brekalo/blob/main/Data_Analytics_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'cyclistic-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4535258%2F7755992%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240305%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240305T123911Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D02b5238c6e0c7ef722c01d1f44a25d73ded95ad35df5ec6080c23afc71dde6cf3ed664b686c483167481bfb1363740875acce1342f1a40d64a3be3ec3a0a66e1f1d804df45a47f705d5a369e0505281076a50659334253c98b148905b14c5a292861c6686fd532baf387cc64ffbe7e0c112b0c373b9151d553812c3094abdf11f895d48b8ee78b2d6ea59a2431ae762dc31572267201114ed9aeac3093237cf1e52ae9c37060c1fd95df04923f86d92a10e2043f3d39a8ccbdd6de6f86f37cb0391a59db99bbcbe411a7b379471443f9b4231edde011c74e9d734ace27015fca6f4affbed078794c04a407128ad17993bf5042995582d53c7527bcb6b72e7d2c'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


<h1 style="color: #272f33;"><strong>Data Analytics Capstone Project: Cyclistic Bike Sharing Analysis</strong></h1>
<font color="#adbac0">Case Study: How Does a Bike-Share Navigate Speedy Success?</font><br>
<font style="color: #272f33;">by Vesna Marija Brekalo</font><br>
<i><font color="#adbac0">February, 27 2024</font></i>

# <font color="#adbac0"><strong>FOREWORD</strong></font>
I completed a case study on **Cyclistic** bike-share for my Google Data Analytics certification, inspired by my passion for cycling. This project, my first in analytics, explores the differences between casual and member riders following Google's framework: **Ask**, **Prepare**, **Process**, **Analyze**, **Share**, **Act**. Check out [my data visualizations here](https://www.linkedin.com/in/vesna-marija-brekalo).

> **Table of Contents:**<br>
<br>
> **INTRODUCTION** <br>
    - Project Overview and Context <br>
> **ASK** <br>
    - Business Goals and Objectives <br>
    - Defining the Analytical Task <br>
    - Identifying Key Stakeholders <br>
> **PREPARE** <br>
    - Data Sourcing and Initial Exploration <br>
> **PROCESS** <br>
    - Data Cleaning and Preprocessin <br>
> **ANALYZE & SHARE** <br>
    - Ride Volume Analysis <br>
    - Average Ride Duration Insights <br>
    - Bike Type Preferences <br>
    - Temporal Usage Patterns <br>
    - Station Popularity Metrics <br>
> **ACT** <br>
    - Strategic Recommendations for Action

# <font color="#adbac0"><strong>INTRODUCTION</strong></font>
**Cyclistic**, founded in 2016, operates a vast fleet of over 16,500 bicycles covering 190 square miles in Chicago. The bikes, strategically located across 800 stations in the city, cater to urban riders with diverse heights and abilities, boasting a design focused on comfort and stability. Users can unlock bikes at any station and return them to any other station at their convenience.

Initially, Cyclistic's marketing aimed at raising general awareness and appealing to a diverse audience with various pricing plans: single-ride passes, full-day passes, and annual memberships. Casual riders are those opting for short-term passes, whereas annual memberships are purchased by Cyclistic members.

Financial analysis indicates annual members generate more profit than casual riders. To capitalize on this, Moreno, Cyclistic's director of marketing, prioritizes converting casual riders into members rather than attracting new customers. The strategy revolves around understanding the differences between these groups, why casual riders might switch to memberships, and the role of digital media in marketing.

Maximizing annual memberships is key to Cyclistic's future success, prompting Moreno's team to analyze historical trip data to uncover trends. As their recommendations necessitate executive approval, they must provide compelling data insights and professional visualizations.

# <font color="#adbac0"><strong>ASK</strong></font>

**Business Objective:**<br>
Develop effective marketing strategies to convert casual riders into annual members. <br>

To **achieve** this **objective**, the future marketing program will address the following three pivotal **questions**:
> 1. How do annual members and casual riders use Cyclistic bikes differently?
> 2. Why would casual riders buy Cyclistic annual memberships?
> 3. How can Cyclistic leverage digital media to encourage casual riders to transition into annual members?

**Key Stakeholders:**
- Lily Moreno: Director of the marketing team and my supervisor.
- Cyclistic executive team.

# <font color="#adbac0"><strong>PREPARE</strong></font>
#### **Data Collection and Sources:**
For this case study, we utilize data from Divvy, a real bike share company operating in Chicago, as a proxy for the fictional company Cyclistic.<br>
These datasets are publicly available through Motivate International Inc. [under a specified license](https://divvybikes.com/data-license-agreement) and can be accessed for [download here](https://divvy-tripdata.s3.amazonaws.com/index.html).<br>
The data is released monthly and is characterized by being anonymized, reliable, original, comprehensive, up-to-date, and properly cited.

In this study, we have analyzed historical travel data from 12 separate CSV files, each corresponding to a month within the period from December 2022 to November 2023. The collective dataset encompasses a total of 5,676,710 recorded trips.

The dataset is structured into 13 distinct columns, or variables, which include:

| No. | Description                            | Column Name      | Type        |
|----:|:----------------------------------------|:------------------|:-------------|
|   1 | Unique identifier for each trip:   | `ride_id`        | String        |
|   2 | Type of bicycle:                     | `rideable_type`  | String        |
|   3 | Start timestamp of the ride:              | `started_at`     | Date & Time |
|   4 | End timestamp of the ride:             | `ended_at`       | Date & Time |
|   5 | Name of the starting station:     | `start_station_name` | String   |
|   6 | Identifier for the starting station:            | `start_station_id`   | String   |
|   7 | Name of the ending station:        | `end_station_name`   | String   |
|   8 | Identifier for the ending station:            | `end_station_id`     | String   |
|   9 | The latitude of the starting station: | `start_lat`          | Numeric|
|  10 | The longitude of starting station:    | `start_lng`          | Numeric|
|  11 | The end station's latitude:           | `end_lat`            | Numeric|
|  12 | The longitude of the end station:     | `end_lng`            | Numeric|
|  13 | Indicates whether the rider is a casual or a member:         | `member_casual`      | String   |

While the dataset did not come with an accompanying metadata file, the variables are intuitively named and can be understood without additional context.

The data categorizes riders into two main groups:
- Casual riders, who typically utilize single rides or day passes
- Cyclistic members, who are characterized by their annual memberships

This categorization is pivotal for understanding different usage patterns and guiding business strategies.

### Divvy [Pricing](https://divvybikes.com/pricing) Summary:
- **Single Ride**: \\$3.30 for up to 30 minutes, with a \\$0.15 per minute overage charge.
- **Daily Pass**: \\$15 for 24-hour access with unlimited classic bike rides up to 3 hours each; overage cost is \\$0.15 per minute for rides over 3 hours.
- **E-Bike Single Ride**: \\$3.30 with an additional \\$0.20 per minute while in use.
- **Annual Membership**: \\$9 monthly for unlimited 45-minute rides; rides exceeding this duration incur a \\$0.16 per minute overage fee.
- **Lost/Stolen Bike Fee**: A penalty of \\$1200.
- **Cycling for Everyone (C4E)**: A program for eligible residents offering a \\$5 annual membership with specific rates and credits for e-bike use.
- **Miscellaneous Fees**: E-bikes can be parked at public racks within the service area for \\$1, with a \\$25 fine for parking violations or if parked outside the service area.

### Tools Utilized for Defining Objectives and Analysis:
- Excel: Employed for the initial data cleaning and review process.
- RStudio Desktop: Used for in-depth data analysis and scripting in R.
- Tableau: Leveraged for creating interactive and informative data visualizations.

The analysis commenced with the meticulous cleaning and scrutiny of the dataset in Excel. Subsequently, RStudio Desktop was utilized to conduct detailed data analysis through R scripting. For the visual representation of the insights, Tableau served as the primary tool, enabling the creation of clear and engaging visualizations.

# <font color="#adbac0"><strong>PROCESS</strong></font>

The data preparation phase was crucial for ensuring the integrity of the subsequent analysis. This phase involved:
1. **Data Cleaning**: Errors, duplicates, blanks, null values, and outliers were meticulously removed to maintain data quality.
2. **Feature Engineering**: New columns were generated for in-depth temporal analysis. These included `starting_at_date`, `starting_at_time`, `ending_at_date`, and `ending_at_time`, extracted from the `started_at` and `ended_at` timestamps. The `ride_length` was recorded in both minutes and seconds, and `day_of_week` was cataloged to map the ride's start day.
3. **Handling Missing Information**: Blanks in `start_station_name` and `end_station_name` were standardized to 'UNKNOWN START STATION NAME' and 'UNKNOWN END STATION NAME', accounting for **2.425%** of the dataset.
4. **Standardizing Station Names**: Anomalies such as additional characters in station names were present in **1.7339%** of rows. These were rectified by removing non-essential characters like asterisks and designations (e.g., '(Temp)', 'Test', city or public rack indicators).
5. **Data Validation**: Rides with illogical durations were excluded, along with those under 2 minutes, which constituted **3.5085%** of the total data.
6. **Reconciling Unknown Stations**: The dataset contained **1791716** cells with 'UNKNOWN' station identifiers, equivalent to **2.4275%** of the data. Utilizing latitude and longitude details, a `list_unique_stations.csv` document facilitated the accurate attribution of station names.

The comprehensive analysis revealed that **5.2533%** of the dataset was incomplete, with **3.5113%** necessitating removal to ensure analytical accuracy.

## Setting up the programming environment

Installing essential packages and libraries for data cleaning and analysis included installing the `tidyverse` package and the `data.table` package, and subsequently, their libraries were also loaded *(details given below)*.

| Load Packages | Description |
|:-----------------|:------------------------------------------------------------------------|
| `library(tidyverse)`  | A collection of R packages that provide versatile tools for data manipulation, visualization, and analysis. |
| `library(dplyr)`      | Specialized in data manipulation, providing a set of functions for efficiently modifying and analyzing data frames. |
| `library(tidyr)`      | Focuses on transforming data to a tidy format, making it more suitable for analysis. |
| `library(readr)`      | Optimized for reading data from various file types, facilitating easy data import. |
| `library(tibble)`     | Enhances data frames, offering a user-friendly approach to data frame creation and manipulation. |
| `library(lubridate)`  | Simplifies working with dates and times, providing functions that make it easier to parse, manipulate, and do calculations with date-time objects. |
| `library(hms)`        | Designed for easy handling and formatting of time data, complementing the `lubridate` package.  |
| `library(purrr)`    | Enhances functional programming in R, offering tools for iterating over data and simplifying complex operations with lists and vectors. It is particularly useful for executing the same function across multiple elements or datasets and working with list-columns in data frames. |
| `library(data.table)` | Offers high-performance data manipulation and analysis, suitable for large datasets. Also, it's highly efficient for exporting data frames to various formats. |
| `library(ggplot2)`     | The `ggplot2` library is a powerful and versatile package for creating data visualizations. |
| `library(scales)` | This library is for formatting the labels. |
| `library(sf)` | The package provides a wide range of functions for analyzing, visualizing, exploring, and working with spatial data. |

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(lubridate)
library(hms)
library(purrr)
library(data.table)
library(ggplot2)
library(scales)
library(sf)

## Unified Consolidation of 12 Monthly Spreadsheets into a Single File, all_rides_df_clean.csv
Utilizing RStudio, I have successfully merged twelve cleansed CSV files ('YYYY_MM_df_update_clean.csv') into one comprehensive dataframe file, all_rides_df_clean.csv.

**An essential step in data analysis is ensuring the data's cleanliness and readiness for further analysis. The following procedures are applied to the `all_rides_df_clean.csv` data frame, each accompanied by a screenshot for visual reference:**
1. **Overview of `all_rides_df_clean.csv`**:
- Obtain an overview of the data frame, including the total number of rows, columns, and cells.

In [ ]:
all_rides_df_clean <- read.csv("/kaggle/input/cyclistic-dataset/all_rides_df_clean.csv")
nrow(all_rides_df_clean) # Number of rows
ncol(all_rides_df_clean) # Number of columns
nrow(all_rides_df_clean) * ncol(all_rides_df_clean) # Number of cells

2. **Previewing Data**:
- Use the `head()` function to display the first few rows of the data frame, providing an initial glimpse into the dataset.

In [ ]:
head(all_rides_df_clean) # View the first few lines

3. **Understanding Data Structure**:
- Examine the data structure and its characteristics, including data types of each column.

In [ ]:
glimpse(all_rides_df_clean) # Data verification

4. **Identifying Outliers**:
- Apply the `summary()` function to summarize each column, aiding in the identification of unusual or outlier values.

In [ ]:
summary(all_rides_df_clean) # Overview of statistics

5. **Missing Values in Data Frame**:
- Check the entire data frame for any missing values to ensure data completeness.

In [ ]:
# Checking for missing values for the entire dataframe
any(is.na(all_rides_df_clean))

6. **Column-wise Missing Values**:
- Investigate missing values by each column to pinpoint areas requiring data imputation or removal.
- Despite missing some values in columns such as end_station_id and start_station_id, these columns, while included, are deemed of lesser importance and not critical to this analysis, and therefore, have been retained in the dataframe.

In [ ]:
library(naniar)
print(miss_var_summary(all_rides_df_clean), n = 25)

7. **Duplicate Rows Check**:
- Verify the presence of duplicate rows within the data frame to maintain data integrity.

In [ ]:
duplicated(all_rides_df_clean)

# <font color="#adbac0"><strong>ANALYZE & SHARE<strong></font>

**Guiding Questions for Enhanced Data Analysis:**

At this phase of our study, we seek to explore the dynamics of bike usage among Cyclistic's annual members versus casual riders. Our analysis will dissect the following key dimensions to uncover distinct patterns of utilization:
- **Ride Volume**: Assessing the total number of rides undertaken by each group.
- **Average Ride Duration**: Calculating the typical length of rides to identify usage intensity.
- **Bike Type Preferences**: Evaluating the preference for different types of bikes between the two groups.
- **Usage Patterns Over Time**: Analyzing how usage varies across different times of day, days of the week, months, and seasons.
- **Popular Stations**: Identifying the most frequented stations to understand spatial preferences.

Through this detailed analysis, we aim to provide insights into the differing behaviors and preferences between annual members and casual riders, contributing to targeted strategy development for Cyclistic.

## Counts and Percentages of Bike Types Used

In [ ]:
# Calculate the number of rides by bike type
ride_counts_by_rideable_type <- table(all_rides_df_clean$rideable_type)

# Calculate the total number of rides
total_rides <- sum(ride_counts_by_rideable_type)

# Calculate the percentage share of each bike type
ride_percentages_rideable_type <- (ride_counts_by_rideable_type / total_rides) * 100

# Create a data frame to display the results
rideable_type_usage <- data.frame(
  Rideable_Type = names(ride_counts_by_rideable_type),
  Ride_Count = as.vector(ride_counts_by_rideable_type),
  Percentage = as.vector(ride_percentages_rideable_type)
)

# Display the results
print(rideable_type_usage)

The number of rides on electric bikes is the highest among the categories, with electric bikes accounting for 2,793,959 rides. Classic bikes are slightly less popular with 2,605,010 rides, while docked bikes have been used significantly less, with only 79,284 rides. This distribution highlights the growing trend towards electric and classic bikes for urban mobility solutions, possibly due to their convenience, ease of access, and efficiency compared to the more stationary docked bikes.

In [ ]:
library(ggplot2)

# Calculate the number of rides by bike type
ride_counts_by_rideable_type <- table(all_rides_df_clean$rideable_type)

# Calculate the total number of rides
total_rides <- sum(ride_counts_by_rideable_type)

# Calculate the percentage share of each bike type
ride_percentages_rideable_type <- (ride_counts_by_rideable_type / total_rides) * 100

# Create a data frame to display the results
rideable_type_usage <- data.frame(
  Rideable_Type = names(ride_counts_by_rideable_type),
  Ride_Count = as.vector(ride_counts_by_rideable_type),
  Percentage = as.vector(ride_percentages_rideable_type)
)

# Convert factor levels to character for labeling
rideable_type_usage$Rideable_Type <- as.character(rideable_type_usage$Rideable_Type)

# Create the pie chart
pie_chart_percentages_of_bike_types_used <- ggplot(
    rideable_type_usage, aes(x = "", y = Percentage, fill = Rideable_Type)) +
  geom_bar(width = 1, stat = "identity") +
  coord_polar("y", start = 0) +
  theme_void() +
  scale_fill_manual(
    values = c("classic_bike" = "#6cc84e",
               "docked_bike" = "#5b79d5",
               "electric_bike" = "#ff468b"),
    name = "Bike Types"
  ) +
  geom_text(
    aes(label = paste0(round(Percentage, 2), "%")),
    position = position_stack(vjust = 0.5)
  ) +
  ggtitle("Percentage distribution of bike rides") +
  labs(subtitle = "The period from 2022-12-01 to 2023-11-30") +
  theme(
    legend.position = "right",
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9)
  )
# Display the pie chart
pie_chart_percentages_of_bike_types_used

Using Cyclistic's historical data on the distribution of bike ride types, the share of electric bikes, classic bikes, and docked bikes is divided into 51.00% for electric bikes, 47.55% for classic bikes, and 1.45% for docked bikes. This data indicates that electric bikes are marginally more popular than classic bikes, with over half of the riders opting for electric bikes.

## Counts and Percentage Distribution of Member and Casual Rides

In [ ]:
# Count rides for 'Member' and 'Casual' in the 'member_casual' column
count_member_casual <- table(all_rides_df_clean$member_casual)

# Display counts of 'Member' and 'Casual'
print("Counts of Member and Casual Rides:")
print(count_member_casual)

# Calculate the percentages for 'Member' and 'Casual'
percentage_member_casual <- prop.table(count_member_casual) * 100

# Display percentages of 'Member' and 'Casual'
print("Percentage of Member and Casual Rides:")
print(percentage_member_casual)

The dataset reveals a significant distinction in usage patterns between annual members and occasional riders within the Cyclistic bike share system.
Members make up the majority of bike rides according to the dataset, with approximately 3,494,248 rides taken by annual members, compared to casual riders who have taken nearly 1,984,005 rides. This suggests that annual members tend to cycle more frequently than casual riders.

In [ ]:
library(ggplot2)

# Count rides for 'Member' and 'Casual' in the 'member_casual' column
count_member_casual <- table(all_rides_df_clean$member_casual)

# Calculate the percentages for 'Member' and 'Casual'
percentage_member_casual <- prop.table(count_member_casual) * 100

# Create a data frame for plotting
member_casual_data <- data.frame(
  Status = names(percentage_member_casual),
  Percentage = as.numeric(percentage_member_casual)
)

# Create the pie chart
pie_chart_percentage_distribution <- ggplot(member_casual_data, aes(x = "", y = Percentage, fill = Status)) +
  geom_bar(stat = "identity", width = 1) +
  coord_polar("y", start = 0) +
  scale_fill_manual(
    values = c("member" = "#01b1c6", "casual" = "#f28e2b"),
    name = "Biker Status"
  ) +
  geom_text(aes(label = paste0(round(Percentage, 2), "%")),
            position = position_stack(vjust = 0.5)) +
  labs(title = "Percentage distribution of Member and Casual Rides",
       subtitle = "The period from 2022-12-01 to 2023-11-30") +
  theme_void() +
  theme(legend.position = "right",
        plot.title = element_text(size = 15, face = "bold"),
        plot.subtitle = element_text(size = 9))

# Display the pie chart
pie_chart_percentage_distribution

Cyclistic's historical data reveals a clear pattern in rider types' usage. Annual members account for the majority of rides at 63.78%, equating to 3,494,248 rides. Casual riders, on the other hand, represent 36.22% of rides, totaling 1,984,005. This data indicates a strong preference for membership-based usage within the Cyclistic bike-sharing community.

## Count of Bike Types Usage by Members and Casual Riders

In [ ]:
# Analyze the use of bike types between members and casual users
usage_by_type_member <- table(all_rides_df_clean$rideable_type
                              [all_rides_df_clean$member_casual == "member"])
usage_by_type_casual <- table(all_rides_df_clean$rideable_type
                              [all_rides_df_clean$member_casual == "casual"])

# Display usage of rideable type by Members
cat("Rideable type usage by Members:\n")
print(usage_by_type_member)

# Display usage of rideable type by Casual riders
cat("Rideable type usage by Casual riders:\n")
print(usage_by_type_casual)

In [ ]:
library(ggplot2)

# Create a summary data frame for the counts of bike type usage by member and casual riders
bike_usage_summary <- aggregate(cbind(
    Number_of_Rides = all_rides_df_clean$rideable_type) ~ rideable_type + member_casual,
                                data = all_rides_df_clean, length)

# Create the bar chart
bar_chart_number_of_bike_types <- ggplot(
    bike_usage_summary, aes(x = rideable_type, y = Number_of_Rides, fill = member_casual)) +
  geom_bar(stat = "identity", position = "dodge") +
  scale_fill_manual(values = c("member" = "#01b1c6", "casual" = "#f28e2b")) +
  labs(title = "Number of Bike Types Usage by Members and Casual Riders",
       subtitle = "The period from 2022-12-01 to 2023-11-30",
       x = "Bike Type",
       y = "Number of Rides",
       fill = "Biker Status") +
  theme_minimal() +
  theme(plot.title = element_text(size = 15, face = "bold"),
        plot.subtitle = element_text(size = 9),
        legend.position = "right")

# Display the bar chart
bar_chart_number_of_bike_types

The graph shows that Cyclistic members predominantly use classic bikes (1,752,793 rides) and electric bikes (1,741,455 rides). Casual riders prefer electric bikes (1,052,504 rides) over classic bikes (852,217 rides) and docked bikes (79,284 rides). Docked bikes are the least utilized type, and their use is exclusive to casual riders within this dataset. Members do not use docked bikes at all.

In [ ]:
# Determine the most commonly used type of bike
most_common_ride_type <- names(sort(table(all_rides_df_clean$rideable_type),
                                    decreasing = TRUE)[1])

# Display the most popular ride type
cat("The most popular ride type is:", most_common_ride_type, "\n")

The **unique** rideable types include electric bikes, classic bikes, docked bikes, and **electric bikes** are the most **popular**.

## Average Ride Length by Users

On average, **members** take shorter rides at **12 minutes** and **29 seconds**, while **casual** riders tend to ride longer, averaging **21 minutes** and **53 seconds**, against an **overall** average ride length of **15 minutes** and **54 seconds** for the year.

In [ ]:
# Required library for time conversion
library(lubridate)

# Direct calculation and formatting of the overall average ride length
mean_ride_length_seconds <- mean(all_rides_df_clean$ride_length_seconds, na.rm = TRUE)
mean_ride_length_hms <- as_hms(mean_ride_length_seconds)
mean_ride_length_formatted <- sprintf("%02d:%02d:%02d",
                                      hour(mean_ride_length_hms),
                                      minute(mean_ride_length_hms),
                                      second(mean_ride_length_hms)
                                     )

# Display the overall average ride length
cat("The overall average length of rides:", mean_ride_length_formatted, "\n")

# Calculation of the average ride length in seconds for members and casual users
average_ride_length_member_seconds <- mean(
    all_rides_df_clean$ride_length_seconds[all_rides_df_clean$member_casual == "member"], na.rm = TRUE)
average_ride_length_casual_seconds <- mean(
    all_rides_df_clean$ride_length_seconds[all_rides_df_clean$member_casual == "casual"], na.rm = TRUE)

# Convert the averages to HH:MM:SS format
average_ride_length_member_hms <- as_hms(average_ride_length_member_seconds)
average_ride_length_member_formatted <- sprintf("%02d:%02d:%02d",
                                                hour(average_ride_length_member_hms),
                                                minute(average_ride_length_member_hms),
                                                second(average_ride_length_member_hms)
                                               )

average_ride_length_casual_hms <- as_hms(average_ride_length_casual_seconds)
average_ride_length_casual_formatted <- sprintf("%02d:%02d:%02d",
                                                hour(average_ride_length_casual_hms),
                                                minute(average_ride_length_casual_hms),
                                                second(average_ride_length_casual_hms)
                                               )

# Display the formatted average ride lengths for member and casual riders
cat("The average ride length for member riders:", average_ride_length_member_formatted, "\n")
cat("The average ride length for casual riders:", average_ride_length_casual_formatted, "\n")

## The most frequent day of the week for rides

**Saturday** is the most popular day for rides **overall** and for **casual** users, while **members** ride most frequently on **Thursdays**.

In [ ]:
# Calculate the mode of the day of the week for all rides
day_of_week_mode <- names(which.max(table(all_rides_df_clean$day_of_week)))

# Display the most common day for all rides
cat("The most frequent day of the week for rides is: ", day_of_week_mode, "\n")

# Calculate the mode of the day of the week specifically for member rides
member_day_of_week_mode <- names(
  which.max(table(all_rides_df_clean$day_of_week[all_rides_df_clean$member_casual == "member"]))
)

# Calculate the mode of the day of the week specifically for casual user rides
casual_day_of_week_mode <- names(
  which.max(table(all_rides_df_clean$day_of_week[all_rides_df_clean$member_casual == "casual"]))
)

# Display the most common day of the week for member rides
cat("The most frequent day of the week for rides for Members: ", member_day_of_week_mode, "\n")

# Display the most common day of the week for casual user rides
cat("The most frequent day of the week for rides for Casual users: ", casual_day_of_week_mode, "\n")

## Calculation of the number of rides by users and bike type usage per day of the week

In [ ]:
# Calculation of the number of rides for all users, members, and casual users by day of the week
num_rides_by_day_of_week <- table(all_rides_df_clean$day_of_week)
num_rides_member_by_day_of_week <- table(
    all_rides_df_clean$day_of_week[all_rides_df_clean$member_casual == "member"])
num_rides_casual_by_day_of_week <- table(
    all_rides_df_clean$day_of_week[all_rides_df_clean$member_casual == "casual"])

# Display results to the console
cat("Number of rides by day of the week for All users:\n")
print(num_rides_by_day_of_week)
cat("Number of rides by day of the week for Members:\n")
print(num_rides_member_by_day_of_week)
cat("Number of rides by day of the week for Casual users:\n")
print(num_rides_casual_by_day_of_week)

# Creating a tibble with the appropriate number of rows (7 for each day of the week)
all_rides_day_week_tibble <- tibble(
  day_of_week = factor(names(num_rides_by_day_of_week), levels = names(num_rides_by_day_of_week)),
  num_rides_users = as.numeric(num_rides_by_day_of_week),
  num_rides_casual = as.numeric(num_rides_casual_by_day_of_week),
  num_rides_member = as.numeric(num_rides_member_by_day_of_week)
)

# Display the created tibble
print(all_rides_day_week_tibble)

# Count of the bike types by day of the week
ride_type_usage_by_day <- table(all_rides_df_clean$day_of_week, all_rides_df_clean$rideable_type)
cat("Ride type usage by Day of the Week:\n")
print(ride_type_usage_by_day)

In [ ]:
# Read CSV file
number_of_rides_by_day_of_week <- read.csv("/kaggle/input/cyclistic-dataset/number_of_rides_by_day_of_week.csv")

# Convert the day_of_week to a factor with the levels in the order you want
number_of_rides_by_day_of_week$day_of_week <- factor(
    number_of_rides_by_day_of_week$day_of_week,
    levels = c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"))

# Custom label function to format the labels as '100K', '200K', etc.
label_100k <- function(x) {
  paste0(x / 1e3, "K")
}

# Plotting the bar chart with ggplot2
rides_plot <- ggplot(number_of_rides_by_day_of_week,
                     aes(x = day_of_week, y = number_of_rides, fill = member_casual)) +
  geom_bar(stat = "identity", position = "dodge") +  # Use dodge for side-by-side bars
  scale_fill_manual(values = c("member" = "#01b1c6", "casual" = "#f28e2b")) +
  labs(title = "Number of User Rides per Day of the Week",
       subtitle = "The period from 2022-12-01 to 2023-11-30",
       x = "Day of the Week",
       y = "Number of Rides",
       fill = "Biker Status") +
  scale_y_continuous(labels = label_100k,
                     breaks = seq(0, max(number_of_rides_by_day_of_week$number_of_rides), by = 100000)) +
  theme_minimal() +
  theme(plot.title = element_text(size = 15, face = "bold"),
        plot.subtitle = element_text(size = 9),
        legend.position = "right",
        axis.text.x = element_text(angle = 45, hjust = 1))

# Display the plot
print(rides_plot)

Based on the graph, Thursday is the peak day for member rides, while Saturday is the most popular for casual users, with these days accounting for 15.13% and 15.41% of weekly rides respectively. Conversely, the least favored days are Sunday for members and Monday for casual users.

## Average Ride Length by Day of the Week for All Users, Members, and Casual Riders

In [ ]:
# Calculate the average ride length for all users by day of the week
average_ride_length_by_day <- all_rides_df_clean %>%
  group_by(day_of_week) %>%
  summarise(ride_length_minutes = mean(ride_length_minutes, na.rm = TRUE))

# Calculate the average ride length for members by day of the week
average_ride_length_member_by_day <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(day_of_week) %>%
  summarise(ride_length_minutes = mean(ride_length_minutes, na.rm = TRUE))

# Calculate the average ride length for casual users by day of the week
average_ride_length_casual_by_day <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(day_of_week) %>%
  summarise(ride_length_minutes = mean(ride_length_minutes, na.rm = TRUE))

# Create a unified data frame for each user type with an additional column for user type
df_all_users <- average_ride_length_by_day %>% mutate(user_type = "All Users")
df_members <- average_ride_length_member_by_day %>% mutate(user_type = "Member")
df_casual <- average_ride_length_casual_by_day %>% mutate(user_type = "Casual")

# Combine all data frames into one
all_rides_df_final_avg_ride_length_by_day <- bind_rows(df_all_users, df_members, df_casual)

# Display the calculated averages to the console with clear separation
cat("Average Ride Length for All Users by Day of the Week:\n")
print(df_all_users)
cat("Average Ride Length for Member Riders by Day of the Week:\n")
print(df_members)
cat("Average Ride Length for Casual Riders by Day of the Week:\n")
print(df_casual)

In [ ]:
# Mapping weekday numbers to day names
day_names <- c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday")

# Adding a new `day_name` column with day names
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(day_name = day_names[day_of_week])

# Calculate the average ride length for members by day of the week
average_ride_length_member_by_day <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(day_name) %>%
  summarise(ride_length_minutes = mean(ride_length_minutes, na.rm = TRUE))

# Calculate the average ride length for casual users by day of the week
average_ride_length_casual_by_day <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(day_name) %>%
  summarise(ride_length_minutes = mean(ride_length_minutes, na.rm = TRUE))

# Create a unified data frame for each user type with an additional column for user type
df_members <- average_ride_length_member_by_day %>% mutate(user_type = "Member")
df_casual <- average_ride_length_casual_by_day %>% mutate(user_type = "Casual")

# Combine all data frames into one
all_rides_df_final_avg_ride_length_by_day <- bind_rows(df_members, df_casual)

# Plotting the bar chart with ggplot2
average_rides_length_plot <- ggplot(all_rides_df_final_avg_ride_length_by_day,
                                    aes(x = day_name, y = ride_length_minutes, fill = user_type)) +
  geom_bar(stat = "identity", position = "stack") +
  scale_fill_manual(values = c("Member" = "#01b1c6", "Casual" = "#f28e2b")) +
  labs(title = "Average Ride Length by Day of the Week for Members and Casual Riders",
       subtitle = "The period from 2022-12-01 to 2023-11-30",
       x = "Day of the Week",
       y = "Duration of the average ride in minutes",
       fill = "User Type") +
  theme_minimal() +
  theme(plot.title = element_text(size = 15, face = "bold"),
        plot.subtitle = element_text(size = 9),
        legend.position = "right",
        axis.text.x = element_text(angle = 45, hjust = 1))

# Display the plot
print(average_rides_length_plot)

The graph illustrates that while members consistently ride for shorter durations across the week, casual riders take longer rides, especially on weekends, which may suggest leisure or tourist activities. Casual riders' trips peak on Sundays, averaging 25.57 minutes, in contrast to members, who average around 12 minutes daily.

## Total Number of Trips for Each User Group during Different Parts of the Day

In [ ]:
# Calculate the total number of rides for each part of the day for all users
total_usage_all <- all_rides_df_clean %>%
  group_by(part_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the total number of rides for each part of the day for members
total_usage_member <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(part_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the total number of rides for each part of the day for casual riders
total_usage_casual <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(part_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Display the results
cat("Total Usage by Part of the Day for All Riders:\n")
print(total_usage_all)
cat("Total Usage by Part of the Day for Members:\n")
print(total_usage_member)
cat("Total Usage by Part of the Day for Casual Riders:\n")
print(total_usage_casual)

In [ ]:
library(scales)

# Total_usage_member in total_usage_casual sta že pripravljena od prejsnjega izracuna
total_usage <- bind_rows(
  total_usage_member %>% mutate(biker_status = "member"),
  total_usage_casual %>% mutate(biker_status = "casual")
)

# Customizing the label design feature
label_format <- function(x) {
  paste0(formatC(x / 1e3, digits = 0, format = "f", big.mark = ","), "K")
}

# Creating a chart
total_usage_by_part_of_the_day_plot <- ggplot(total_usage,
                                              aes(x = part_of_day, y = total_rides, fill = biker_status)) +
  geom_bar(stat = "identity", position = "dodge") +
  scale_fill_manual(values = c("casual" = "#f28e2b", "member" = "#01b1c6")) +
  scale_y_continuous(
    labels = label_format,
    breaks = seq(0, 1200000, by = 200000),
    limits = c(NA, 1200000)
  ) +
  labs(
    title = "Total Number of Trips during Different Parts of the Day",
    subtitle = "The period from 2022-12-01 to 2023-11-30",
    x = "Part of Day",
    y = "Number of Rides",
    fill = "Biker status"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9),
    legend.position = "right",
    axis.text.x = element_text(angle = 45, hjust = 1)
  )

# # Display graph
print(total_usage_by_part_of_the_day_plot)

The chart shows that the afternoon is the most popular time for bike rides, with member trips evenly distributed throughout morning, afternoon, and evening, while casual riders exhibit a peak in the afternoon.

## Top 10 Most Popular Riding Hours

In [ ]:
# Calculate the hour from the 'started_at_time' column, which is in the HH:MM:SS string format
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(hour_of_day = as.integer(substr(started_at_time, 1, 2)))

# Calculate the number of rides per hour for all users
hour_of_day_usage_all_top <- all_rides_df_clean %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop') %>%
  top_n(10, total_rides)

# Calculate the number of rides per hour for members
hour_of_day_usage_member_top <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop') %>%
  top_n(10, total_rides)

# Calculate the number of rides per hour for casual riders
hour_of_day_usage_casual_top <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop') %>%
  top_n(10, total_rides)

# Print the top 10 most popular riding hours for all users
cat("Top 10 Most Popular Riding Hours for All Users:\n")
print(hour_of_day_usage_all_top %>% arrange(desc(total_rides)))

# Print the top 10 most popular riding hours for members
cat("Top 10 Most Popular Riding Hours for Members:\n")
print(hour_of_day_usage_member_top %>% arrange(desc(total_rides)))

# Print the top 10 most popular riding hours for casual riders
cat("Top 10 Most Popular Riding Hours for Casual Riders:\n")
print(hour_of_day_usage_casual_top %>% arrange(desc(total_rides)))

In [ ]:
# Create a graph to show the most popular cycling hours for all users
most_popular_riding_hours <- ggplot(hour_of_day_usage_all_top, aes(x = hour_of_day, y = total_rides)) +
  geom_line(group = 1, color = "#948473") +
  geom_point(size = 3, color = "#f8048e") +
  scale_y_continuous(
    labels = scales::label_number(scale = 1e-3, suffix = "K"),
    breaks = seq(0, 600 * 1e3, by = 100 * 1e3),
    limits = c(0, 600 * 1e3)
  ) +
  scale_x_continuous(breaks = hour_of_day_usage_all_top$hour_of_day) +
  labs(
    title = "Top 10 Most Popular Riding Hours Overall",
    subtitle = "The period from 2022-12-01 to 2023-11-30",
    x = "Hour of Day AM/PM",
    y = "Number of Rides"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9)
  )

# Display graph
print(most_popular_riding_hours)

The data from the chart indicates that the late afternoon, specifically around 5 PM, is the most popular time for cycling, with 564,221 rides occurring during this time. Furthermore, the time span from 3 PM to 6 PM is the preferred window for cycling, accounting for 1,905,528 rides, which represents 34.7835% of total rides.

In [ ]:
# Calculate the popularity of rides between 3 PM and 6 PM to determine
# the number of rides in this time frame in both absolute and relative terms compared to all rides.

all_rides_df_clean <- all_rides_df_clean %>%
  mutate(hour_of_day = as.integer(substr(started_at_time, 1, 2)))

# Filter rides between 3 PM and 6 PM
hourly_usage_peak <- all_rides_df_clean %>%
  filter(hour_of_day >= 15 & hour_of_day <= 18)

# Calculate the total number of rides in this time period
total_rides_peak <- nrow(hourly_usage_peak)

# Calculate the total number of rides in the entire dataset for comparison
total_rides_all <- nrow(all_rides_df_clean)

# Calculate the share of rides in this time period compared to all rides
percentage_peak <- (total_rides_peak / total_rides_all) * 100

# Display the results
cat("Total Rides between 3 PM and 6 PM:", total_rides_peak, "\n")
cat("Percentage of Total Rides:", format(percentage_peak, nsmall = 2), "%\n")

In comparison with the previous graph, the following analysis is performed:
- The peak cycling hours are between **3 PM** and **6 PM**, with **1,905,528** total rides, comprising 34.78% of all rides.
- Evening rides, from **6 PM** to **11 PM**, total **1,464,636**, making up 26.74% of rides, indicating a significant preference for post-work cycling.
- Late-night to early morning hours, from **12 AM** to **5 AM**, have the fewest rides at **210,012**, accounting for only 3.83% of the total, reflecting the least popular time for cycling.
- Early to mid-morning rides, from **5 AM** to **8 AM**, total **714,303**, which is 13.04% of rides, showing a moderate uptake for early risers or commuters.
- Late morning to midday, from **8 AM** to **12 PM**, sees **1,342,721** rides, comprising 24.51% of the total, indicating a preference for mid-morning cycling.
- Combining early and late morning periods, from **5 AM** to **12 PM**, yields a total of 1,754,320 rides, or 32.02% of total rides, suggesting morning hours are fairly popular for cycling.
- From noon to **3 PM**, there are **1,352,599** total rides, amounting to 24.69% of the total, which suggests a steady use of bikes into the early afternoon.
- Extending from early morning through to the afternoon, from **5 AM** to **3 PM**, there are a total of **2,790,682** rides, representing a substantial 50.94% of all rides, indicating that the daytime hours up until mid-afternoon are the most popular for cycling.

## Calculation of the Number of Rides During Different Times of the Day

In [ ]:
# Calculate the hour from the timestamp
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(hour_of_day = hour(hms::as_hms(started_at_time)))

# Calculate the number of rides per hour for all users
hour_of_day_usage_all <- all_rides_df_clean %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the number of rides per hour for members
hour_of_day_usage_member <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the number of rides per hour for casual riders
hour_of_day_usage_casual <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Display the results
cat("Hourly Usage for All Riders:\n")
print(hour_of_day_usage_all)
cat("Hourly Usage for Members:\n")
print(hour_of_day_usage_member)
cat("Hourly Usage for Casual Riders:\n")
print(hour_of_day_usage_casual)

In [ ]:
# Create a graph to show all cycling hours for all users
rides_different_times_of_the_Day <- ggplot(hour_of_day_usage_all, aes(x = hour_of_day, y = total_rides)) +
  geom_line(group = 1, color = "#948473") +
  geom_point(size = 3, color = "#f8048e") +
  scale_y_continuous(
    labels = scales::label_number(scale = 1e-3, suffix = "K"),
    breaks = seq(0, 600 * 1e3, by = 100 * 1e3),
    limits = c(0, 600 * 1e3)
  ) +
  scale_x_continuous(breaks = hour_of_day_usage_all$hour_of_day) +
  labs(
    title = "Number of Rides During Different Times of the Day",
    subtitle = "The period from 2022-12-01 to 2023-11-30",
    x = "Hour of Day AM/PM",
    y = "Number of Rides"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9)
  )

# Display graph
print(rides_different_times_of_the_Day)

The graph highlights that the period from afternoon to evening is the most favored for cycling, with the hours between 3 PM and 6 PM alone accounting for 34.78% of daily rides, signifying that over a third of the day's rides occur in this 3-hour window. Morning hours show the least activity, and there is a noticeable decline in rides from 6 PM to 11 PM, which constitutes 26.74% of the daily rides.

## Percentage Distribution of Total Ride Counts by Hour of Day

The graph's percentage distribution indicates that 5 PM is the peak time for rides, accounting for 10.30% of the total. There is a significant drop in activity from 11 PM to 5 AM, marking these as the least popular hours. Cyclistic users clearly favor the time span between 3 PM and 6 PM for their rides.

In [ ]:
# Calculate the hour from the timestamp
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(hour_of_day = hour(hms::as_hms(started_at_time)))

# Calculate the number of rides per hour for all users
hour_of_day_usage_all <- all_rides_df_clean %>%
  group_by(hour_of_day) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the percentages for each hour of day
hour_of_day_usage_all <- hour_of_day_usage_all %>%
  mutate(percentage_rides = (total_rides / sum(total_rides)) * 100)

# Create the graph for the distribution of rides by hour of day
percentage_total_ride_by_hour_of_day <- ggplot(hour_of_day_usage_all,
                                               aes(x = factor(hour_of_day), y = percentage_rides)) +
  geom_bar(stat = "identity", fill = "#a5998c") +
  scale_y_continuous(
    labels = function(x) paste0(x, "%"),
    breaks = seq(0, 12, by = 2),
    limits = c(0, 12)
  ) +
  scale_x_discrete(
    name = "Hour of Day AM/PM",
    labels = function(x) {
      am_pm <- ifelse(as.numeric(x) < 12, "AM", "PM")
      hour <- as.numeric(x) %% 12
      hour[hour == 0] <- 12
      paste0(hour, " ", am_pm)
    }
  ) +
  labs(
    title = "Percentage Distribution of Total Ride Counts by Hour of Day",
    subtitle = "The period from 2022-12-01 to 2023-11-30",
    y = "Percentage of Rides",
    x = "Hour of Day"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9)
  )

# Display graph
print(percentage_total_ride_by_hour_of_day)

In [ ]:
# Calculate the hour from the timestamp
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(hour_of_day = hour(hms::as_hms(started_at_time)))

# Calculate the number of rides per hour for members and casual riders and combine them
hour_of_day_usage <- all_rides_df_clean %>%
  group_by(hour_of_day, member_casual) %>%
  summarize(total_rides = n(), .groups = 'drop')


# Create the graph
total_ride_by_hour_of_day_member_casual <- ggplot(
    hour_of_day_usage, aes(x = factor(hour_of_day), y = total_rides, fill = member_casual)) +
  geom_bar(stat = "identity", position = position_dodge()) +
  scale_fill_manual(values = c("casual" = "#f28e2b", "member" = "#01b1c6")) +
  scale_y_continuous(
    labels = label_number(scale = 1e-3, suffix = "K"),
    breaks = seq(0, 400000, by = 50000),
    limits = c(0, 400000)
  ) +
  scale_x_discrete(
    name = "Hour of Day AM/PM",
    labels = function(x) {
      am_pm <- ifelse(as.numeric(x) < 12, "AM", "PM")
      hour <- as.numeric(x) %% 12
      hour[hour == 0] <- 12
      paste0(hour, " ", am_pm)
    }
  ) +
  labs(
    title = "Total Ride Counts by Hour of Day for Casual Riders and Members",
    subtitle = "The period from 2022-12-01 to 2023-11-30",
    x = "Hour of Day AM/PM",
    y = "Number of Rides",
    fill = "Biker Status"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9),
    legend.position = "right",
    axis.text.x = element_text(angle = 45, hjust = 1)
  )

# Display graph
print(total_ride_by_hour_of_day_member_casual)

The graph compares the hourly ride counts of casual riders and members, showing that members tend to ride mostly during morning peak hours, with a significant spike around 8 AM, and again in the late afternoon, peaking at 5 PM. Casual riders, on the other hand, have their highest counts later in the day, with their peak also at 5 PM but sustaining higher ride counts into the evening compared to members.

## Number of Bike Rides and Percentage Distribution Across Seasons

In [ ]:
# Convert 'started_at_date' and 'ended_at_date' columns to date format
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(
    started_at_date = as.Date(started_at_date),
    ended_at_date = as.Date(ended_at_date)
  )

# Create a new 'season' column to label the seasons
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(
    season = case_when(
      between(started_at_date, as.Date("2023-03-21"), as.Date("2023-06-20")) ~ "Spring",
      between(started_at_date, as.Date("2023-06-21"), as.Date("2023-09-22")) ~ "Summer",
      between(started_at_date, as.Date("2023-09-23"), as.Date("2023-12-20")) ~ "Autumn",
      TRUE ~ "Winter"
    )
  )

# Calculate the number of rides for each season
rides_by_season <- all_rides_df_clean %>%
  group_by(season) %>%
  summarize(Count_of_ride_id = n(), .groups = 'drop')

# Print the results for rides by season
cat("Number of rides per season:\n")
print(rides_by_season)


# Calculate the number of rides for each season, segmented by 'member_casual' column
rides_by_season_member_casual <- all_rides_df_clean %>%
  group_by(season, member_casual) %>%
  summarize(Count_of_ride_id = n(), .groups = 'drop')

# Print the results for rides by season and user type
cat("Number of rides per season and user type:\n")
print(rides_by_season_member_casual)

In [ ]:
library(ggplot2)

# Classification in order of seasons
rides_by_season$season <- factor(rides_by_season$season,
                                 levels = c("Winter", "Spring", "Summer", "Autumn"))

# Plotting the bar chart
season_graph <- ggplot(rides_by_season, aes(x = season, y = Count_of_ride_id, fill = season)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = c(
    "Winter" = "#7bd1fc", "Spring" = "#9fea74", "Summer" = "#fee37a", "Autumn" = "#aaa9f7")) +
  scale_y_continuous(
    labels = scales::label_number(scale = 1e-3, suffix = "K")
  ) +
  labs(
    title = "Number of Bike Rides per Season",
    subtitle = "The period from 2022-12-01 to 2023-11-30",
    x = "",
    y = "Number of Rides"
    # fill = "Season"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9),
    legend.position = "right",
    axis.text.x = element_text(angle = 0, hjust = 1),
    legend.title = element_blank()
  )

# Display graph
print(season_graph)

The bar chart shows the number of bike rides per season, indicating that summer is the most popular season for biking with a total of 2,192,634 rides. Spring follows with 1,547,735 rides, then autumn with 1,048,806 rides, and winter has the least with 689,078 rides. This suggests that warmer weather significantly increases the frequency of bike usage.

In [ ]:
# Calculate the total number of rides
total_rides <- sum(rides_by_season$Count_of_ride_id)

# Add a column for the percentage
rides_by_season <- rides_by_season %>%
  mutate(Percentage = (Count_of_ride_id / total_rides) * 100)

# Create a pie chart
season_pia <- ggplot(rides_by_season, aes(x = "", y = Percentage, fill = season)) +
  geom_bar(stat = "identity", width = 1) +
  coord_polar("y", start = 0) +
  scale_fill_manual(values = c(
    "Winter" = "#7bd1fc", "Spring" = "#9fea74", "Summer" = "#fee37a", "Autumn" = "#aaa9f7")) +
  theme_void() +
  labs(
    title = "Percentage Distribution of Bike Rides Across Seasons",
    subtitle = "The period from 2022-12-01 to 2023-11-30"
  ) +
  geom_text(aes(label = sprintf("%s\n%.2f%%", season, Percentage)),
            position = position_stack(vjust = 0.5)
  ) +
  # theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9),
    legend.position = "right",
    legend.title = element_blank()
  )

# Display graph
print(season_pia)

The pie chart clearly demonstrates that summer is the most popular season for cycling, with a total of 2,192,634 rides, accounting for 40.02% of the season's total rides. Following summer, spring is the next favored season with 1,547,735 rides, making up 28.25% of the rides. These figures indicate a strong seasonal trend in cycling habits, with riders significantly preferring the warmer months, which can be clearly seen in the calculation below.

## Bike Ride Count and Percentage Distribution per Month for Member and Casual Riders

In [ ]:
# Convert 'started_at' column to date format
all_rides_df_clean$started_at <- as.Date(all_rides_df_clean$started_at)

# Calculation of months from the 'started_at' column
all_rides_df_clean$month <- format(all_rides_df_clean$started_at, "%B")

# Calculate the number of rides per month for all users
ride_counts_by_month_all <- all_rides_df_clean %>%
  group_by(month) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the number of rides per month for members
ride_counts_by_month_member <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(month) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Calculate the number of rides per month for casual riders
ride_counts_by_month_casual <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(month) %>%
  summarize(total_rides = n(), .groups = 'drop')

# Display the results
cat("Ride Counts by Month for All Riders:\n")
print(ride_counts_by_month_all)
cat("Ride Counts by Month for Members:\n")
print(ride_counts_by_month_member)
cat("Ride Counts by Month for Casual Riders:\n")
print(ride_counts_by_month_casual)

In [ ]:
# Convert 'started_at' column to date format
all_rides_df_clean$started_at <- as.Date(all_rides_df_clean$started_at)

# Adding a month column with the month name
all_rides_df_clean$month <- month.name[month(all_rides_df_clean$started_at)]

# Create a summary data frame with counts for each month and member status
monthly_rides <- all_rides_df_clean %>%
  count(month, member_casual)

# Use ggplot to create the stacked bar chart
bike_rides_by_month <- ggplot(monthly_rides, aes(x = month, y = n, fill = member_casual)) +
  geom_bar(stat = "identity", position = "stack") +
  labs(title = "Number of Bike Rides by Month for Member and Casual Riders",
       subtitle = "The period from 2022-12-01 to 2023-11-30",
       x = "Month",
       y = "Number of Rides",
       fill = "Biker Status") +  # Correct place for legend title
  scale_fill_manual(values = c("casual" = "#f28e2b", "member" = "#01b1c6")) +
  scale_y_continuous(
    labels = scales::label_number(scale = 1e-3, suffix = "K"),
    breaks = seq(0, 800000, by = 100000),
    limits = c(0, 800000)
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9),
    legend.position = "bottom",
    axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 8)
  )

# Display graph
print(bike_rides_by_month)

The bar chart confirms that August is the month with the highest number of bike rides, followed closely by July, and then June, indicating a peak in cycling activity during the summer months. Conversely, the winter months of December, January, and February have the fewest recorded rides, reflecting a seasonal decline in cycling frequency.

## Number of Unique Starting and End Stations
* Number of unique **starting** stations: **1542**

In [ ]:
# Calculate the number of unique starting stations in the dataset
num_start_stations <- length(unique(all_rides_df_clean$start_station_name))
# Print the result for the number of unique starting stations
cat("Number of unique starting stations:", num_start_stations, "\n")

## Most common used Start station by day of the week
* The most common Start station: **Old Hastings Charging Stx**
* The **total** number of all latitudes and longitudes for **'Old Hastings Charging Stx'** stations: **810968**
* The **total** number of **unique** latitudes and longitudes for 'Old Hastings Charging Stx' stations: **731**

In [ ]:
# Identifying the most commonly used starting station
most_common_start_station <- names(sort(table(all_rides_df_clean$start_station_name),
                                        decreasing = TRUE)[1])
cat("Most common Start station:", most_common_start_station, "\n")

# Filtering data for stations named "Old Hastings Charging Stx"
old_hastings_stations_all <- all_rides_df_clean %>%
  filter(start_station_name == "Old Hastings Charging Stx") %>%
  select(start_lng, start_lat)

# Selecting unique combinations of geographical latitudes
# and longitudes for stations named "Old Hastings Charging Stx"
old_hastings_stations_unique <- all_rides_df_clean %>%
  filter(start_station_name == "Old Hastings Charging Stx") %>%
  distinct(start_lng, start_lat)

# Counting all occurrences of "Old Hastings Charging Stx" stations
total_all_old_hastings_stations <- nrow(old_hastings_stations_all)

# Counting all unique "Old Hastings Charging Stx" stations based on their latitudes and longitudes
total_unique_old_hastings_stations <- nrow(old_hastings_stations_unique)

# Displaying the total number of all and unique "Old Hastings Charging Stx" stations
cat("Total number of all entries for 'Old Hastings Charging Stx' stations:",
    total_all_old_hastings_stations, "\n")
cat("Number of unique entries for 'Old Hastings Charging Stx' stations based on latitudes and longitudes:",
    total_unique_old_hastings_stations, "\n")

## Top 20 Start Stations for Members, Casual Riders, and All Users

In [ ]:
# Identify the top 20 most frequently used start stations for members
top_stations_member <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  group_by(start_station_name) %>%
  summarise(count = n()) %>%
  arrange(desc(count)) %>%
  slice_head(n = 20)

# Display the top 20 start stations for members
cat("Top 20 Start Stations for Members:\n")
print(top_stations_member)

# Identify the top 20 most frequently used start stations for casual riders
top_stations_casual <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  group_by(start_station_name) %>%
  summarise(count = n()) %>%
  arrange(desc(count)) %>%
  slice_head(n = 20)

# Display the top 20 start stations for casual riders
cat("Top 20 Start Stations for Casual Riders:\n")
print(top_stations_casual)

# Identify the top 20 most frequently used start stations regardless of user type
top_stations_all <- all_rides_df_clean %>%
  group_by(start_station_name) %>%
  summarise(count = n()) %>%
  arrange(desc(count)) %>%
  slice_head(n = 20)

# Display the top 20 start stations overall
cat("Top 20 Start Stations Overall:\n")
print(top_stations_all)

In [ ]:
# Calculate the top 20 starting stations for all users
top_stations_all <- all_rides_df_clean %>%
  group_by(start_station_name) %>%
  summarise(count = n(),
            latitude = first(start_lat),
            longitude = first(start_lng)) %>%
  ungroup() %>%
  arrange(desc(count)) %>%
  top_n(20, wt = count) %>%  # Select top 20 stations by count
  mutate(full_latitude = latitude,  # Add new column for full latitude
         full_longitude = longitude)  # Add new column for full longitude

# Add a label for top 20 stations
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(top20 = if_else(start_station_name %in% top_stations_all$start_station_name, 'Top 20', 'Others'))

# Create a map with ggplot2
top_20_start_stations <- ggplot(data = all_rides_df_clean) +
  geom_point(aes(x = start_lng, y = start_lat, color = top20), size = 1) +
  scale_color_manual(values = c("#76b7b2", "#ff0657")) +
  labs(title = "Top 20 Starting Stations for All Users",
       subtitle = "The period from 2022-12-01 to 2023-11-30",
       x = "Longitude", y = "Latitude") + # Add labels for longitude and latitude
  coord_sf() +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 15, face = "bold"),
    plot.subtitle = element_text(size = 9),
    legend.position = "right"
  )

# Display map
print(top_20_start_stations)

The map showcases the top 20 starting stations for bike rides, with Old Hastings Charging Stx standing out due to its exceptionally high number of rides, totaling 810,968. This station appears to be an outlier, as its usage significantly exceeds that of other stations. However, this may be due to the station's name being associated with multiple locations across the city. Therefore, while it appears prominently in the dataset, its data might be misleading and should be interpreted with caution when considering station popularity and usage patterns.

## Most Popular Stations by Time of Day: Members, Casual Riders, and All Users

In [ ]:
# Function to determine the part of the day
part_of_day <- function(time) {
  hour <- hour(as.POSIXct(time, format = "%H:%M:%S"))
  if (hour >= 5 & hour < 12) {
    return("Morning")
  } else if (hour >= 12 & hour < 17) {
    return("Afternoon")
  } else if (hour >= 17 & hour < 21) {
    return("Evening")
  } else {
    return("Night")
  }
}

# Calculate the part of the day for each ride using sapply
all_rides_df_clean <- all_rides_df_clean %>%
  mutate(part_of_day = sapply(started_at_time, part_of_day))

# Function for analysis by part of the day
analyse_usage <- function(data, top_stations) {
  data %>%
    filter(start_station_name %in% top_stations) %>%
    count(start_station_name, part_of_day) %>%
    arrange(start_station_name, part_of_day)
}

# Identify the top 20 stations for all users
top_stations_all <- all_rides_df_clean %>%
  count(start_station_name) %>%
  arrange(desc(n)) %>%
  slice_head(n = 20) %>%
  pull(start_station_name)

# Identify the top 20 stations for members
top_stations_member <- all_rides_df_clean %>%
  filter(member_casual == "member") %>%
  count(start_station_name) %>%
  arrange(desc(n)) %>%
  slice_head(n = 20) %>%
  pull(start_station_name)

# Identify the top 20 stations for casual riders
top_stations_casual <- all_rides_df_clean %>%
  filter(member_casual == "casual") %>%
  count(start_station_name) %>%
  arrange(desc(n)) %>%
  slice_head(n = 20) %>%
  pull(start_station_name)

# Use the analyse_usage function for each user group
hourly_usage_all <- analyse_usage(all_rides_df_clean, top_stations_all)
hourly_usage_member <- analyse_usage(
    all_rides_df_clean %>% filter(member_casual == "member"), top_stations_member)
hourly_usage_casual <- analyse_usage(
    all_rides_df_clean %>% filter(member_casual == "casual"), top_stations_casual)

# Display and save the data
cat("Usage by Part of the Day for Top 20 Stations for All Users:\n")
print(hourly_usage_all)
cat("Usage by Part of the Day for Top 20 Stations for Members:\n")
print(hourly_usage_member)
cat("Usage by Part of the Day for Top 20 Stations for Casual Riders:\n")
print(hourly_usage_casual)

The calculation shows the most popular bike rental stations segmented by time of day and user type. It reveals that Old Hastings Charging Stx is the top station across all times of day for both casual riders and members, indicating its central role in the network. Notably, there's a trend showing that certain stations are favored during specific times, such as Streeter Dr & Grand Ave during both morning and night, suggesting these locations may be hubs of activity at the beginning and end of the day. This data can inform decisions on station maintenance, bike supply allocation, and targeted marketing strategies to balance demand throughout the day.

## Summary Revision:
**Members:**
- Members' cycling peaks during commuting hours on weekdays, with a surge from 7 am to 9 am and another from 3 pm to 6 pm, aligning with work-related travel patterns. This trend is most pronounced during the warmer seasons of summer and early fall, as well as spring.
- The data shows members have a higher frequency of rides but with shorter average durations, typically half that of casual riders' journeys.
- Common starting and ending points for members are closely linked to universities, residential neighborhoods, and business districts, suggesting a correlation with daily routines and work or academic commitments.

**Casual Riders:**
- Casual riders demonstrate a preference for midday to early evening rides, peaking from 11 am to 7 pm, with weekends being particularly popular. This pattern is prevalent during leisure-friendly warmer months, such as summer and early fall.
- Their rides are generally longer in duration, about twice as long as those of members, indicating a leisurely or exploratory approach to cycling.
- Preferred locations for casual riders to pick up and drop off bikes include scenic parks, tourist hotspots, museums, waterfront areas, and other places of leisure, underscoring their recreational use of the bike-share system.

# <font color="#adbac0"><strong>ACT</strong></font>
## Revised Recommendations
Based on data insights, here are refined strategies for Cyclistic to convert casual riders into annual members:

### Understanding User Dynamics:
1. **Usage Pattern Analysis**:
   - Investigate the duration, frequency, and timing of bike usage by casual riders compared to members, focusing on peak usage times for targeted promotions.
2. **Membership Value Proposition**:
   - Research what drives casual riders to rent bikes and craft a compelling value proposition for the annual membership that addresses their specific needs and motivations.
3. **Digital Engagement**:
   - Analyze the digital touchpoints where casual riders interact with Cyclistic and develop a digital engagement strategy to nurture their journey towards membership.

### Tactical Steps for Membership Growth:
1. **Tailored Promotions**:
   - Launch special weekday pricing and loyalty rewards aimed at casual riders to incentivize regular usage and familiarity with the service.
2. **Subscription Flexibility**:
   - Offer a range of subscription plans including weekly or monthly passes to serve as stepping stones to full annual membership.
3. **Focused Marketing Efforts**:
   - Target marketing efforts in areas with high casual rider traffic and on platforms popular with tourists, emphasizing the benefits of membership over single-use passes.

### Engagement and Pricing Strategies:
1. **Incentive-Based Pricing**:
   - Introduce a tiered pricing model that encourages longer-term commitments, with clear cost benefits over casual usage.
2. **Membership Perks**:
   - Provide exclusive benefits for members, such as priority bike access during high demand periods or flexible pass options for varying rider needs.
3. **Smart Ad Placement**:
   - Deploy targeted ads in high-traffic locations and during peak seasonal periods to maximize exposure to potential members.
4. **Cost-Savings Messaging**:
   - Highlight the long-term savings and convenience of an annual membership, particularly in comparison to the cumulative cost of multiple casual rentals.

### Strategic Analysis and Market Adaptation:
- **Early Data Utilization**:
  - Prioritize the analysis of emerging trends and the exploration of new data analysis tools to capture nuanced user behaviors.
- **Data-Driven Model Adaptation**:
  - Collect more granular data to understand the preferences of local, occasional riders and adapt the membership model accordingly.
- **Membership Model Innovation**:
  - Re-evaluate the membership offerings to align with the lifestyle and cycling patterns of Chicago's occasional riders, ensuring the model remains competitive and appealing.
  
> *Thank you for taking the time to review my project.<br>
> I sincerely appreciate your attention and am eagerly looking forward to your feedback, critiques, or suggestions.*